In [62]:
RANDOM_SEED = 42

In [63]:
import os

# 현재 스크립트 파일의 디렉토리 경로 가져오기
script_dir = os.path.dirname(r'C:\Users\pop75\OneDrive\바탕 화면\Qunova')

# 하위 폴더의 경로 설정 (예: subfolder)
subfolder = 'subfolder'

# 하위 폴더의 절대 경로 생성
subfolder_path = os.path.join(script_dir, subfolder)

# 환경 변수 PATH에 하위 폴더 경로 추가
os.environ['PATH'] += os.pathsep + subfolder_path

In [64]:
import pandas as pd
from TDC.tdc import utils as tdc_utils
from TDC.tdc.single_pred import Tox
import RDK as rk
from DeepPurpose import utils as dp_utils, CompoundPred
import numpy as np

## Prepare Data(TDC package, herg)

In [115]:
import random
X, y = Tox(name = 'carcinogens_lagunin').get_data(format = 'DeepPurpose')
## get 50000 molecule
Data_num = 6000
Drug_info_plus, Tox_info_plus = X[y==1][:Data_num], y[y==1][:Data_num]
Drug_info_minus, Tox_info_minus = X[y==0][:Data_num], y[y==0][:Data_num]
Drug_info = np.array(list(Drug_info_plus)+list(Drug_info_minus))
Tox_info = np.array(list(Tox_info_plus)+list(Tox_info_minus))

Found local copy...
Loading...
Done!


In [116]:
## Change SMILE to RDkit fingerprint
Drug_fin = rk.smile_to_RDkit(Drug_info,bits=2048)

In [73]:
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
dim = 4
pca = PCA(n_components=dim)
Drug_16 = pca.fit_transform(Drug_fin)
smote = SMOTE()
Drug_16_smote,Tox_info_smote = smote.fit_resample(Drug_16, Tox_info)

In [68]:
from sklearn.model_selection import train_test_split  
train_features, test_features, train_labels, test_labels = train_test_split(Drug_16_smote,Tox_info_smote, test_size=0.25, random_state=42)

## Run with logistic regresion

In [69]:
from sklearn.linear_model import LogisticRegression  
from imblearn.over_sampling import SMOTE
from imblearn.combine import *
from sklearn.decomposition import PCA

model = LogisticRegression()
model.fit(train_features, train_labels)
accuracy = model.score(test_features, test_labels)
train_accuracy = model.score(train_features, train_labels)
print(f"Test set accuracy: {accuracy}, Train set accuracy : {train_accuracy}")

Test set accuracy: 0.7454545454545455, Train set accuracy : 0.8151515151515152


In [70]:
from sklearn.metrics import roc_curve, roc_auc_score, auc
import matplotlib.pyplot as plt
rk.Result(model,Drug_16_smote,Tox_info_smote,plot=False)

--------------------0 KFold-------------------
train_idx_len : 330 / test_idx_len : 110
AUC: 0.869
Test set accuracy: 0.7818181818181819, Train set accuracy : 0.793939393939394
precision : 0.8076923076923077
recall : 0.75
f1_score : 0.7777777777777779
--------------------1 KFold-------------------
train_idx_len : 330 / test_idx_len : 110
AUC: 0.898
Test set accuracy: 0.8272727272727273, Train set accuracy : 0.803030303030303
precision : 0.7878787878787878
recall : 0.9122807017543859
f1_score : 0.8455284552845528
--------------------2 KFold-------------------
train_idx_len : 330 / test_idx_len : 110
AUC: 0.857
Test set accuracy: 0.7909090909090909, Train set accuracy : 0.806060606060606
precision : 0.7540983606557377
recall : 0.8518518518518519
f1_score : 0.7999999999999999
--------------------3 KFold-------------------
train_idx_len : 330 / test_idx_len : 110
AUC: 0.893
Test set accuracy: 0.8, Train set accuracy : 0.796969696969697
precision : 0.7924528301886793
recall : 0.792452830188

## Run with Neural network

In [71]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
SPLITS = 4
kf = KFold(n_splits = SPLITS,shuffle=True)
n_iter = 0
mlp = MLPClassifier(hidden_layer_sizes=(100,30), max_iter=12000, random_state=42,learning_rate='constant',alpha=0.01)
mlp.fit(train_features,train_labels)
#scaler = StandardScaler()  
accuracy = mlp.score(test_features, test_labels)
train_accuracy = mlp.score(train_features, train_labels)
print(f"Test set accuracy: {accuracy}, Train set accuracy : {train_accuracy}")

Test set accuracy: 0.8727272727272727, Train set accuracy : 0.9848484848484849


In [72]:
rk.Result(mlp,Drug_16_smote,Tox_info_smote,plot=False)

--------------------0 KFold-------------------
train_idx_len : 330 / test_idx_len : 110
AUC: 0.930
Test set accuracy: 0.8818181818181818, Train set accuracy : 0.9878787878787879
precision : 0.8571428571428571
recall : 0.9523809523809523
f1_score : 0.9022556390977443
--------------------1 KFold-------------------
train_idx_len : 330 / test_idx_len : 110
AUC: 0.944
Test set accuracy: 0.8818181818181818, Train set accuracy : 0.9818181818181818
precision : 0.8703703703703703
recall : 0.8867924528301887
f1_score : 0.8785046728971964
--------------------2 KFold-------------------
train_idx_len : 330 / test_idx_len : 110
AUC: 0.944
Test set accuracy: 0.8545454545454545, Train set accuracy : 0.9939393939393939
precision : 0.8235294117647058
recall : 0.8571428571428571
f1_score : 0.84
--------------------3 KFold-------------------
train_idx_len : 330 / test_idx_len : 110
AUC: 0.961
Test set accuracy: 0.9181818181818182, Train set accuracy : 0.9878787878787879
precision : 0.8833333333333333
reca

## Quantum kernal train

In [74]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

## Make the feature map
ZZ_feature_map = ZZFeatureMap(feature_dimension=dim, reps=1, entanglement="linear")

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)
## K(a,b)
q_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=ZZ_feature_map)
## Pre evaluatre the inner product value
q_matrix_train = q_kernel.evaluate(x_vec=train_features)
q_matrix_test = q_kernel.evaluate(x_vec=test_features, y_vec=train_features)

In [114]:
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn import metrics

q_svc = SVC(kernel="precomputed")

q_svc.fit(q_matrix_train, train_labels)

accuracy = q_svc.score(q_matrix_test, test_labels)

train_accuracy = q_svc.score(q_matrix_train , train_labels)

print(f"Quantum result Test set accuracy: {accuracy}, Train set accuracy : {train_accuracy}")

y_scores = q_svc.predict(q_matrix_train)

print("\nTrain Result")
print("accuracy: ", metrics.accuracy_score(train_labels, y_scores))
print("precision: ", metrics.precision_score(train_labels, y_scores)) 
print("recall: ", metrics.recall_score(train_labels, y_scores))
print("f1: ", metrics.f1_score(train_labels, y_scores))
print("area under curve (auc): ", metrics.roc_auc_score(train_labels, y_scores))

y_scores = q_svc.predict(q_matrix_test)

print("\nTest Result")
print("accuracy: ", metrics.accuracy_score(test_labels, y_scores))
print("precision: ", metrics.precision_score(test_labels, y_scores)) 
print("recall: ", metrics.recall_score(test_labels, y_scores))
print("f1: ", metrics.f1_score(test_labels, y_scores))
print("area under curve (auc): ", metrics.roc_auc_score(test_labels, y_scores))

Quantum result Test set accuracy: 0.7272727272727273, Train set accuracy : 0.8848484848484849

Train Result
accuracy:  0.8848484848484849
precision:  0.8711656441717791
recall:  0.8930817610062893
f1:  0.8819875776397514
area under curve (auc):  0.8851373717312149

Test Result
accuracy:  0.7272727272727273
precision:  0.7627118644067796
recall:  0.7377049180327869
f1:  0.7500000000000001
area under curve (auc):  0.7259953161592506


In [111]:
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn import metrics

linear_svc = SVC(kernel="linear")

linear_svc.fit(train_features, train_labels)

accuracy = linear_svc.score(test_features, test_labels)

train_accuracy = linear_svc.score(train_features , train_labels)


print(f"Test set accuracy: {accuracy}, Train set accuracy : {train_accuracy}")

y_scores = linear_svc.predict(train_features)

print("\nTrain Result")
print("accuracy: ", metrics.accuracy_score(train_labels, y_scores))
print("precision: ", metrics.precision_score(train_labels, y_scores)) 
print("recall: ", metrics.recall_score(train_labels, y_scores))
print("f1: ", metrics.f1_score(train_labels, y_scores))
print("area under curve (auc): ", metrics.roc_auc_score(train_labels, y_scores))

y_scores = linear_svc.predict(test_features)

print("\nTest Result")
print("accuracy: ", metrics.accuracy_score(test_labels, y_scores))
print("precision: ", metrics.precision_score(test_labels, y_scores)) 
print("recall: ", metrics.recall_score(test_labels, y_scores))
print("f1: ", metrics.f1_score(test_labels, y_scores))
print("area under curve (auc): ", metrics.roc_auc_score(test_labels, y_scores))

Test set accuracy: 0.7272727272727273, Train set accuracy : 0.8090909090909091

Train Result
accuracy:  0.8090909090909091
precision:  0.7474226804123711
recall:  0.9119496855345912
f1:  0.821529745042493
area under curve (auc):  0.812699988966126

Test Result
accuracy:  0.7272727272727273
precision:  0.7384615384615385
recall:  0.7868852459016393
f1:  0.7619047619047619
area under curve (auc):  0.7199732351957178


In [113]:
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn import metrics

rbf_svc = SVC(kernel="rbf")

rbf_svc.fit(train_features, train_labels)

accuracy = rbf_svc.score(test_features, test_labels)

train_accuracy = rbf_svc.score(train_features , train_labels)


print(f"Test set accuracy: {accuracy}, Train set accuracy : {train_accuracy}")

y_scores = rbf_svc.predict(train_features)

print("\nTrain Result")
print("accuracy: ", metrics.accuracy_score(train_labels, y_scores))
print("precision: ", metrics.precision_score(train_labels, y_scores)) 
print("recall: ", metrics.recall_score(train_labels, y_scores))
print("f1: ", metrics.f1_score(train_labels, y_scores))
print("area under curve (auc): ", metrics.roc_auc_score(train_labels, y_scores))

y_scores = rbf_svc.predict(test_features)

print("\nTest Result")
print("accuracy: ", metrics.accuracy_score(test_labels, y_scores))
print("precision: ", metrics.precision_score(test_labels, y_scores)) 
print("recall: ", metrics.recall_score(test_labels, y_scores))
print("f1: ", metrics.f1_score(test_labels, y_scores))
print("area under curve (auc): ", metrics.roc_auc_score(test_labels, y_scores))

Test set accuracy: 0.7727272727272727, Train set accuracy : 0.8636363636363636

Train Result
accuracy:  0.8636363636363636
precision:  0.8166666666666667
recall:  0.9245283018867925
f1:  0.8672566371681416
area under curve (auc):  0.8657729228732208

Test Result
accuracy:  0.7727272727272727
precision:  0.78125
recall:  0.819672131147541
f1:  0.8
area under curve (auc):  0.7669789227166277
